In [7]:
import cdsapi
import pandas as pd
import xarray as xr
from shapely import geometry
from pathlib import Path
from typing import Union
from datetime import datetime

In [9]:
ts < datetime.now()

False

In [13]:
def download_era(date: pd.Timestamp, out_dir: Union[str, Path], subset: Union[geometry.Polygon, None]) -> str:
    """
    Download era5 weather model for specific hourly timestep as netcdf. Can be subset to a specific geographic subset.

    Args:
    date: timestamp of the desired date and time. Will be rounded to nearest hour
    out_dir: directory to save file into
    subset: subset geometry to clip data to

    returns:
    out_fp: filepath of saved netcdf
    """
    # check to see if era5 token has been saved to .cdsapric file and create client for download
    assert Path('~/.cdsapirc').expanduser().exists(), "Must sign up for ERA5 account and save key to .cdsapric file"
    c = cdsapi.Client()

    out_dir = out_dir.expanduser()
    assert out_dir.exists(), f"Provided directory: {out_dir} does not exist."

    # could get 'relative_humidity' but sticking with one.
    humidparam = 'specific_humidity'

    # pressure levels to download (this is all 37 possible)
    era_pressure_lvls = ['1','2','3','5','7','10','20','30','50', '70','100','125',\
    '150','175','200','225', '250','300','350','400','450','500','550','600','650',\
    '700','750','775','800','825', '850','875','900','925','950','975','1000']

    assert date < datetime.now(), "provided date is in the future"
    assert date > pd.to_datetime('1940-01-01'), "provided date is before january 1940"

    indict = {'product_type'   :'reanalysis',\
                'format'         :'netcdf',\
                'variable'       :['geopotential','temperature', humidparam],\
                'pressure_level' : era_pressure_lvls, \
                'date'           : date.strftime('%Y-%M-%d'),
                'time'           : date.strftime('%H:00')}

    out_fp = out_dir.joinpath(f"ERA5_{date.strftime('%Y-%M-%dT%H:00')}.nc")

    if subset:
        w, s, e, n = subset.bounds
        assert w > -180 and w < 180, f"West bound: {w} is outside of globe"
        assert e > -180 and e < 180, f"East bound: {e} is outside of globe"
        assert n > -90 and n < 90, f"North bound: {n} is outside of globe"
        assert s > -90 and w < 90, f"South bound: {s} is outside of globe"
        indict['area'] = f'/'.join([str(b) for b in [n, w, s, e]])
        out_fp = out_fp.with_stem(f"{out_fp.stem}_{'_'.join([str(b) for b in subset.bounds])}")

    c.retrieve('reanalysis-era5-pressure-levels', indict, target=out_fp)

    return out_fp

In [14]:
area = geometry.box(-116, 91, -115, 46)
ts = pd.to_datetime('2020-01-03T20:01:0000')
out_dir = Path('/tmp').expanduser()

out_fp = download_era(date = ts, out_dir = out_dir, subset = area)

ds = xr.open_dataset(out_fp)


AssertionError: North bound: 91.0 is outside of globe

In [6]:
ts = pd.to_datetime('2024-01-03T20:01:0000')
out_fp = download_era(date = ts, out_dir = out_dir, subset = area)


2023-05-04 11:24:41,083 INFO Welcome to the CDS
2023-05-04 11:24:41,083 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2023-05-04 11:24:41,296 INFO Request is queued
2023-05-04 11:24:42,474 INFO Request is failed
2023-05-04 11:24:42,474 ERROR Message: the request you have submitted is not valid
2023-05-04 11:24:42,474 ERROR Reason:  None of the data you have requested is available yet, please revise the period requested. The latest date available for this dataset is: 2023-04-29 17:24:41.905039
2023-05-04 11:24:42,475 ERROR   Traceback (most recent call last):
2023-05-04 11:24:42,475 ERROR     File "/opt/cdstoolbox/cdscompute/cdscompute/cdshandlers/services/handler.py", line 59, in handle_request
2023-05-04 11:24:42,475 ERROR       result = cached(context.method, proc, context, context.args, context.kwargs)
2023-05-04 11:24:42,475 ERROR     File "/opt/cdstoolbox/cdscompute/cdscompute/caching.py", line 108, in cached
2023-05-04 

Exception: the request you have submitted is not valid. None of the data you have requested is available yet, please revise the period requested. The latest date available for this dataset is: 2023-04-29 17:24:41.905039.